El objetivo de este código es implementar el algoritmo GBM, separando los datos por repetición.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [ ]:
# !pip install comet_ml

# Importo las librerías
from lightgbm import LGBMClassifier
import numpy as np
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import time
from comet_ml import Experiment
import joblib
import matplotlib.pyplot as plt

# Levanto los datos
features = load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50.joblib")
label = load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib")

Voy a dividir los datos a mano de la siguiente forma.
Un sujeto repite cada gesto entre 5 y 6 veces. Voy a usar:
- Una repetición para test.
- Una para validacion.
- Las restantes para train.

En este sentido, voy a guardar los indices de label en listas distintas, para identificar luego que filas de FEATURES tengo que usar para cada conjunto de datos (train, val y test).

La implementación de esta idea parte del supuesto de que los gestos iguales estan agrupados, es decir, las repeticiones del mismo gesto son consecutivas. Esto para los gestos que no son reposo.

Para los gestos de reposo se va a hacer un trato diferenciado. Primero se va a hacer la partición sin ellos, y luego, cada repetición de reposo se va a incluir en el mismo conjunto de la repetición de gesto previa a ese reposo.

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"

/content/drive/My Drive/repo_tesis/archivos_generados_codigos


El siguiente bloque se código genera una partición de los datos por repetición. Si ya se ejecutó en la parte de mlp, se pueden usar las mismas.

In [ ]:
for k in range(1, 11):
  label = np.array(label)
  ind_reposo = np.where(label[:, 1]==0)[0]
  ind_gestos = np.where(label[:, 1]!=0)[0]
  label_gestos = label[ind_gestos, :]
  division_gestos = np.where(np.diff(label_gestos[:, 1])!=0)[0]
  division_id_ventanas = np.where(np.diff(label_gestos[:, 0])!=0)[0]
  train = []
  val = []
  test = []
  for i in range(len(division_gestos)+1):  #
      if i==len(division_gestos):
          ind_tope_ventana = np.max(label_gestos[:, 0])
          ind_inicio_ventana = label_gestos[division_gestos[-1], 0]
          cant_ventanas = ind_tope_ventana - ind_inicio_ventana
      else:
          tope = division_gestos[i]   # este es limite con el proximo gesto

          if i==0:
              inicio = 0
              ind_inicio_ventana = 0
          else:
              inicio = division_gestos[i-1]
              ind_inicio_ventana = np.where(division_id_ventanas==inicio)[0][0]
          ind_tope_ventana = np.where(division_id_ventanas==tope)[0][0]
          cant_ventanas = ind_tope_ventana - ind_inicio_ventana
      if i != 0:
          sorteo = np.random.permutation(cant_ventanas)
          sorteo += 1     # correccion de rango
      else:
          sorteo = np.random.permutation(cant_ventanas+1)
      test.append(ind_inicio_ventana + sorteo[0])
      val.append(ind_inicio_ventana + sorteo[1])
      for j in range(2, len(sorteo)):
          train.append(ind_inicio_ventana + sorteo[j])
  indices_train = []
  for j in range(len(train)):
      indices_train.extend(np.where(label[:, 0]==train[j])[0])

  indices_val = []
  indices_test = []
  for j in range(len(val)):
      indices_val.extend(np.where(label[:, 0]==val[j])[0])
      indices_test.extend(np.where(label[:, 0]==test[j])[0])
  div_repe_reposo = np.where(np.diff(ind_reposo)!=1)[0]
  div_repe_reposo = np.insert(div_repe_reposo, 0, -1)   # el -1 es porque todos me apuntan al ultimo elemento de cada repeticion, en el for le voy a sumar 1 a todos para que me apunten al inicio que es lo que me interesa

  # Calculo cantidad de repeticiones de reposo
  cant_repeticiones_reposo = len(div_repe_reposo)

  pertenencia_conjuntos = []  # train, val, test. Cada dato solo puede pertenecer a 1 de estos conjuntos

  for i in range(len(div_repe_reposo)):
      ind0_repe_reposo_act = ind_reposo[div_repe_reposo[i]+1]
      id_repe_act = label[ind0_repe_reposo_act, 0]
      id_repe_ant = label[ind0_repe_reposo_act-1, 0]

      pertenencia_conjuntos.append([id_repe_act, id_repe_ant in train, id_repe_ant in val, id_repe_ant in test])

  pertenencia_conjuntos = np.array(pertenencia_conjuntos)

  id_reposo_train = pertenencia_conjuntos[np.where(pertenencia_conjuntos[:, 1])[0], 0]
  id_reposo_val = pertenencia_conjuntos[np.where(pertenencia_conjuntos[:, 2])[0], 0]
  id_reposo_test = pertenencia_conjuntos[np.where(pertenencia_conjuntos[:, 3])[0], 0]

  for j in range(len(id_reposo_train)):
      indices_train.extend(np.where(label[:, 0]==id_reposo_train[j])[0])

  for j in range(len(id_reposo_val)):
      indices_val.extend(np.where(label[:, 0]==id_reposo_val[j])[0])
      indices_test.extend(np.where(label[:, 0]==id_reposo_test[j])[0])
  joblib.dump(indices_train, 'indices_train_' + str(k) + '.pkl')
  joblib.dump(indices_val, 'indices_val_' + str(k) + '.pkl')
  joblib.dump(indices_test, 'indices_test_' + str(k) + '.pkl')

En el siguiente código se implementa GBM con el split generado.

In [3]:
%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"
for k in range(1, 11):
  features = np.array(features)
  label = np.array(label)

  indices_train = load('indices_train_' + str(k) + '.pkl')
  #X_train = features[indices_train, :]
  #y_train = label[indices_train, 1]
  #del(indices_train)

  indices_val = load('indices_val_' + str(k) + '.pkl')
  #X_val = features[indices_val, :]
  #y_val = label[indices_val, 1]
  #del(indices_val)

  indices_test = load('indices_test_' + str(k) + '.pkl')
  #X_test =  features[indices_test, :]
  #y_test = label[indices_test, 1]
  #del(indices_test)

  start_time = time.time()

  # Borrar todo lo que no use de aqui para adelante en este punto

  # implemento gbm
  clf = LGBMClassifier()
  clf.fit(features[indices_train, :], label[indices_train, 1], eval_set=[(features[indices_val, :], label[indices_val, 1])], eval_metric='logloss')

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo
  bst = clf.best_iteration_

  nombre = 'baseline_gbm_sep_rep' + str(k) + '_testing'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(features[indices_test, :])

  accuracy = accuracy_score(label[indices_test, 1], y_pred)
  precision = precision_score(label[indices_test, 1], y_pred, average='weighted')
  recall = recall_score(label[indices_test, 1], y_pred, average='weighted')
  conf_matrix = confusion_matrix(label[indices_test, 1], y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'gbm', 'sep_rep', 'choose_test']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(label[indices_test, 1], y_pred)
  exp.log_parameter("ind_train", indices_train)
  exp.log_parameter("ind_val", indices_val)
  exp.log_parameter("ind_test", indices_test)
  exp.log_text("Experimento separando los datos por repeticiones y usando GBM. En cuanto a las ventanas: W=200, I=50.")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

Output hidden; open in https://colab.research.google.com to view.